In [4]:
!pip insta keras
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

ERROR: unknown command "update"

Using TensorFlow backend.


ImportError: cannot import name 'transpose_shape' from 'keras.utils.generic_utils' (C:\Users\anand\Anaconda3\lib\site-packages\keras\utils\generic_utils.py)

In [ ]:
batch_size = 128
n_inputs = 784
n_classes = 10
n_epochs = 20

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.reshape(60000, n_inputs)
x_test = x_test.reshape(10000, n_inputs)

In [ ]:
# convert the input values to float32
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
# normalize the values of image vectors to fit under 1
x_train /= 255
x_test /= 255
# convert output data into one hot encoded format
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

In [2]:
# build a sequential model
model = Sequential()
# the first layer has to specify the dimensions of the input vector
model.add(Dense(units=512, activation='relu', input_shape=(n_inputs,)))
# add dropout layer for preventing overfitting
model.add(Dropout(0.1))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))
# output layer can only have the neurons equal to the number of outputs
model.add(Dense(units=n_classes, activation='softmax'))
# print the summary of our model
model.summary()
# compile the model
model.compile(loss='categorical_crossentropy',
 optimizer="Adam",
 metrics=['accuracy'])
# train the model
model.fit(x_train, y_train,
 batch_size=batch_size,
 epochs=n_epochs,
 shuffle=True)

NameError: name 'Sequential' is not defined

In [ ]:
# evaluate the model and print the accuracy score
scores = model.evaluate(x_test, y_test)
print('\n loss:', scores[0])
print('\n accuracy:', scores[1])
print(model.summary)

In [ ]:
model.save('mnist2.h5')
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file('mnist2.h5')
tflite_model = converter.convert()
open('mnist2.tflite', 'wb').write(tflite_model)

In [ ]:
try:
    from google.colab import files
    files.download('mnist2.tflite')
except:
    print("Skip downloading")

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import os
import tensorflow as tf
mnist = input_data.read_data_sets(os.path.join('.','mnist'), one_hot=True)
X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels

In [ ]:
n_classes = 10 # 0-9 digits
n_width = 28
n_height = 28
n_depth = 1
n_inputs = n_height * n_width * n_depth # total pixels
learning_rate = 0.001
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

In [ ]:
# input images shape: (n_samples,n_pixels)
x = tf.placeholder(dtype=tf.float32, name="x", shape=[None,
n_inputs])
# output labels
y = tf.placeholder(dtype=tf.float32, name="y", shape=[None,
n_classes])
x_ = tf.reshape(x, shape=[-1, n_width, n_height, n_depth])

In [ ]:
layer1_w = tf.Variable(tf.random_normal(shape=[4,4,n_depth,32],
 stddev=0.1),name='l1_w')
layer1_b = tf.Variable(tf.random_normal([32]),name='l1_b')
layer1_conv = tf.nn.relu(tf.nn.conv2d(x_,layer1_w,
 strides=[1,1,1,1],
 padding='SAME'
 ) +
 layer1_b
 )
layer1_pool = tf.nn.max_pool(layer1_conv,ksize=[1,2,2,1],
 strides=[1,2,2,1],padding='SAME')


layer2_w = tf.Variable(tf.random_normal(shape=[4,4,32,64],
 stddev=0.1),name='l2_w')
layer2_b = tf.Variable(tf.random_normal([64]),name='l2_b')
layer2_conv = tf.nn.relu(tf.nn.conv2d(layer1_pool,
 layer2_w,
 strides=[1,1,1,1],
 padding='SAME'
 ) +
 layer2_b
 )
layer2_pool = tf.nn.max_pool(layer2_conv,
 ksize=[1,2,2,1],
 strides=[1,2,2,1],
 padding='SAME'
 )

layer3_w = tf.Variable(tf.random_normal(shape=[64*7*7*1,1024],
 stddev=0.1),name='l3_w')
layer3_b = tf.Variable(tf.random_normal([1024]),name='l3_b')
layer3_fc = tf.nn.relu(tf.matmul(tf.reshape(layer2_pool,
 [-1, 64*7*7*1]),layer3_w) + layer3_b)
layer4_w = tf.Variable(tf.random_normal(shape=[1024, n_classes],
 stddev=0.1),name='l')
layer4_b = tf.Variable(tf.random_normal([n_classes]),name='l4_b')
layer4_out = tf.matmul(layer3_fc,layer4_w) + layer4_b
                       
model = layer4_out

In [ ]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [ ]:
with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        total_loss = 0.0
        for batch in range(n_batches):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            feed_dict={x:batch_x, y: batch_y}
            batch_loss,_ = tfs.run([loss, optimizer],
            feed_dict=feed_dict)
            total_loss += batch_loss
        average_loss = total_loss / n_batches
        print("Epoch: {0:04d} loss = {1:0.6f}".format(epoch,average_loss))
print("Model Trained.")
predictions_check = tf.equal(tf.argmax(model,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(predictions_check, tf.float32))
feed_dict = {x:mnist.test.images, y:mnist.test.labels}
print("Accuracy:", accuracy.eval(feed_dict=feed_dict))